In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam
import pandas as pd
import numpy as np
import os

Using TensorFlow backend.


In [15]:
def load_model(filename, verbose=False):
    json_file = open(filename + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(filename + ".h5")
    if verbose:
        print("The following model " + filename + " has been loaded")
    return loaded_model

## Load data

In [4]:
DATA_FOLDER = "./data/"

In [5]:
df_training = pd.read_parquet(DATA_FOLDER + "training.parquet")

In [6]:
df_public_test = pd.read_parquet(DATA_FOLDER + "public_test.parquet")

In [7]:
df_private_test = pd.read_parquet(DATA_FOLDER + "private_test.parquet")

## Process data

In [8]:
x_train = df_training.as_matrix()[:,1:]
y_train = df_training.as_matrix()[:,0]
x_test = df_public_test.as_matrix()[:,1:]
y_test = df_public_test.as_matrix()[:,0]
x_private_test = df_private_test.as_matrix()[:,1:]
y_private_test = df_private_test.as_matrix()[:,0]

c:\python3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
c:\python3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
c:\python3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
c:\python3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
c:\python3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
c:\python3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be

In [9]:
x_train = x_train.reshape(-1, 48, 48, 1)
x_test = x_test.reshape(-1, 48, 48, 1)
x_private_test = x_private_test.reshape(-1, 48, 48, 1)

In [10]:
x_train = x_train.astype("float32")/255.
x_test = x_test.astype("float32")/255.
x_private_test = x_private_test.astype("float32")/255.

In [11]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_private_test = to_categorical(y_private_test)

## Test model

In [3]:
filename = "trained_models/first_model"

In [12]:
model = load_model(filename)

The following model trained_models/first_model has been loaded


In [13]:
model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-4), metrics=["accuracy"])

In [35]:
def evaluate(model, x_train, y_train, x_test, y_test, x_private_test, y_private_test):
    
    loss, acc = model.evaluate(x_train, y_train, batch_size=1000, verbose=0)
    print("Train")
    print("loss: " + "{0:.2f}".format(loss) + " acc: " + "{0:.2f}".format(acc))
    
    loss, acc = model.evaluate(x_test, y_test, batch_size=1000, verbose=0)
    print("Test")
    print("loss: " + "{0:.2f}".format(loss) + " acc: " + "{0:.2f}".format(acc))
    
    
    loss, acc = model.evaluate(x_private_test, y_private_test, batch_size=1000, verbose=0)
    print("Private test")
    print("loss: " + "{0:.2f}".format(loss) + " acc: " + "{0:.2f}".format(acc))
    

In [ ]:
evaluate(model, x_train, y_train, x_test, y_test, x_private_test, y_private_test)

In [33]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_private_test, model.predict(x_private_test), labels=None)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets